## <font color=yellow>1. 동영상 요약</font>

- vertex ai 
- 2분 51초 동영상 요약

In [1]:
# import os
import json
import time
import base64

import streamlit as st
import vertexai

from vertexai.generative_models import GenerativeModel, Part

In [17]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# from IPython.display import Video
# Video(video_path, width=600, height=400)

In [4]:
video_path = "./data/news.mp4"

In [5]:
def summarize(video):
    model = GenerativeModel("gemini-2.5-pro")
    responses = model.generate_content(
        [video, "이 영상을 bullet point 사용해서 3줄로 요약해줘"],
        generation_config={
            "max_output_tokens": 2048,
            "temperature": 0.4,
            "top_p": 1,
            "top_k": 32
        },
    )
    return responses.text

In [6]:
with open(video_path, "rb") as f:
    video_encoded = base64.b64encode(f.read())

video = Part.from_data(data=video_encoded.decode('utf-8'), mime_type="video/mp4")
text = summarize(video)

c:\Users\jjsd4\workspace\torch\Lib\site-packages\vertexai\generative_models\_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [7]:
print(text)

다음은 해당 영상의 내용을 세 줄로 요약한 것입니다.

*   한 금융 분석 보고서(UBS)는 엔비디아의 주당 순이익이 41달러에 이를 것으로 전망하며, 이는 현재 주가가 저평가되었을 수 있음을 시사합니다.
*   차세대 칩인 '블랙웰(Blackwell)'은 기존의 텍스트 기반 AI를 넘어, AI가 비디오를 학습하고 생성하는 능력을 갖추게 하는 핵심 기술로 평가됩니다.
*   이러한 비디오 처리 기술의 발전은 로봇이 영상을 보고 스스로 학습하는 등 AI 응용 분야의 새로운 가능성을 열어줄 것으로 기대됩니다.


## <font color=yellow>2. 동영상 구간 별 요약하기</font>
- 동영상이 길 경우 이렇게 사용
- 또는 구간 별로 요약해서 섹션별로 요약된 내용을 볼 수 있음

In [8]:
from moviepy import VideoFileClip
import tempfile
import os

In [9]:
TEMP_DIR = "./tmp_video_segments"

In [10]:
def ensure_temp_dir():
    if not os.path.exists(TEMP_DIR):
        os.makedirs(TEMP_DIR)
        
def process_video(video_path, segment_duration=60, overlap_duration=10):
    ensure_temp_dir()
    clip = VideoFileClip(video_path)
    total_duration = clip.duration
    segments = []
    temp_files = []

    for i, start_time in enumerate(range(0, int(total_duration), segment_duration - overlap_duration)):
        end_time = min(start_time + segment_duration, total_duration)
        segment = clip.subclipped(start_time, end_time) # overlap 있게 코드를 자름
        
        temp_file_path = os.path.join(TEMP_DIR, f"segment_{i}.mp4") # 파일 이름
        segment.write_videofile(temp_file_path, codec="libx264") # segment를 파일로 저장
        temp_files.append(temp_file_path)
        
        with open(temp_file_path, "rb") as f: # gemini로 보내려면 인코딩이 필요
            video_encoded = base64.b64encode(f.read()).decode('utf-8')
        video_part = Part.from_data(data=video_encoded, mime_type="video/mp4") # part 형식으로 바꿔야함
        segments.append((start_time, end_time, video_part))
        
        if end_time == total_duration:
            break

    clip.close()
    return segments, temp_files

In [11]:
def summarize_segment(video_segment, previous_summary="", segment_info=""):
    model = GenerativeModel("gemini-2.5-pro")
    context = f"이전 구간 요약: {previous_summary}\n\n현재 구간 정보: {segment_info}\n\n"
    prompt = context + "이 영상 구간을 앞선 맥락을 고려하여 bullet point 사용해서 2-3줄로 요약해줘. 중복되는 내용은 제외하고, 새로운 정보나 변화에 초점을 맞춰주세요."
    
    responses = model.generate_content(
        [video_segment, prompt],
        generation_config={
            "max_output_tokens": 2048,
            "temperature": 0.4,
            "top_p": 1,
            "top_k": 32
        },
    )
    return responses.text

In [12]:
def summarize_video(video_path, segment_duration=60, overlap_duration=10):
    segments, temp_files = process_video(video_path, segment_duration, overlap_duration)
    summaries = []
    previous_summary = ""

    try:
        for i, (start_time, end_time, segment) in enumerate(segments):
            segment_info = f"구간 {i+1} ({start_time:.1f}s - {end_time:.1f}s)"
            summary = summarize_segment(segment, previous_summary, segment_info)
            summaries.append(f"{segment_info}:\n{summary}\n")
            previous_summary = summary  # 다음 요약을 위해 현재 요약을 저장

        # 전체 요약 생성
        full_summary = "\n".join(summaries)
        model = GenerativeModel("gemini-2.5-pro")
        final_summary = model.generate_content(
            [f"다음은 비디오의 각 구간별 요약입니다. 이를 바탕으로 전체 비디오의 내용을 5개의 bullet point로 요약해주세요. 중복을 피하고 주요 내용과 변화에 초점을 맞춰주세요:\n\n{full_summary}"],
            generation_config={
                "max_output_tokens": 2048,
                "temperature": 0.4,
                "top_p": 1,
                "top_k": 32
            },
        ).text

        return final_summary, summaries

    finally:
        # 모든 처리가 끝난 후 임시 파일 삭제
        for temp_file in temp_files:
            try:
                os.remove(temp_file)
            except Exception as e:
                print(f"임시 파일 삭제 중 오류 발생: {e}")
        
        # 임시 디렉토리가 비어있으면 삭제
        if not os.listdir(TEMP_DIR):
            os.rmdir(TEMP_DIR)

In [13]:
video_path = "./data/news.mp4"
final_summary, summaries = summarize_video(video_path, segment_duration=60, overlap_duration=10)

MoviePy - Building video ./tmp_video_segments\segment_0.mp4.
MoviePy - Writing audio in segment_0TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
MoviePy - Writing video ./tmp_video_segments\segment_0.mp4



MoviePy - Done !
MoviePy - video ready ./tmp_video_segments\segment_0.mp4
MoviePy - Building video ./tmp_video_segments\segment_1.mp4.
MoviePy - Writing audio in segment_1TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
MoviePy - Writing video ./tmp_video_segments\segment_1.mp4



MoviePy - Done !
MoviePy - video ready ./tmp_video_segments\segment_1.mp4
MoviePy - Building video ./tmp_video_segments\segment_2.mp4.
MoviePy - Writing audio in segment_2TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
MoviePy - Writing video ./tmp_video_segments\segment_2.mp4



MoviePy - Done !
MoviePy - video ready ./tmp_video_segments\segment_2.mp4
MoviePy - Building video ./tmp_video_segments\segment_3.mp4.
MoviePy - Writing audio in segment_3TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
MoviePy - Writing video ./tmp_video_segments\segment_3.mp4



c:\Users\jjsd4\workspace\torch\Lib\site-packages\vertexai\generative_models\_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


MoviePy - Done !
MoviePy - video ready ./tmp_video_segments\segment_3.mp4


In [14]:
print("\n\n".join(summaries))

구간 1 (0.0s - 60.0s):
다음은 해당 영상 구간에 대한 요약입니다.

*   한 분석가는 UBS 보고서를 인용하며, 엔비디아(NVIDIA)의 주당 순이익(EPS)이 41달러에 이를 것으로 예상되어 현재 주가가 "저렴하다"고 주장합니다.
*   그는 시장의 우려와 달리 차세대 칩인 "블랙웰(Blackwell)"이 예상보다 일찍 출시될 것이라고 예측합니다.
*   블랙웰 칩은 기존의 텍스트 기반 AI를 넘어 비디오를 처리하고 생성하는 능력을 갖추게 될 것이며, 이는 AI 기술의 중대한 발전을 의미한다고 강조합니다.


구간 2 (50.0s - 110.0s):
다음은 해당 영상 구간에 대한 요약입니다.

*   분석가는 AI가 비디오를 "섭취(ingest)"하고 학습하는 능력이 "차세대 개척지"가 될 것이라고 설명합니다. 예를 들어, AI가 영화를 보고 대니얼 크레이그나 캐리 그랜트 같은 배우의 행동과 스타일을 복제할 수 있게 됩니다.
*   이러한 비디오 기반 학습은 로봇 공학의 발전에 핵심적인 역할을 할 것입니다. 로봇이 단순한 텍스트 명령을 넘어, 관찰을 통해 정교하고 인간과 유사한 행동을 학습할 수 있게 되기 때문입니다.


구간 3 (100.0s - 160.0s):
다음은 해당 영상 구간에 대한 요약입니다.

*   분석가는 AI의 발전이 로봇의 상호작용 방식을 근본적으로 바꿀 것이라고 예측합니다. 현재의 기계적인 로봇과 달리, 미래의 로봇은 "어이, 파트너, 잘 지내?"와 같이 자연스럽고 인간적인 대화를 나눌 수 있게 될 것입니다.
*   이러한 기술은 단순히 작업을 수행하는 것을 넘어, 개인화되고 상황에 맞는 서비스를 제공하는 데 활용될 것입니다. 예를 들어, AI 바텐더는 사용자의 과거 주문이나 특정 기념일(예: 싱코 데 마요)을 기억하여 그에 맞는 음료를 추천할 수 있습니다.
*   NVIDIA의 CEO조차 이 기술의 모든 잠재적 활용 사례를 알지 못한다고 언급하며, 개발자들에게 도구를 제공하여 혁신을 이끌어내는 것이 중요하다고 강조합니

In [15]:
print(final_summary)

네, 제공된 구간별 요약을 바탕으로 영상 전체의 핵심 내용을 5개의 bullet point로 정리해 드리겠습니다.

*   한 분석가는 엔비디아의 차세대 칩 '블랙웰'의 조기 출시와 높은 예상 수익을 근거로 현재 주가가 저평가되었다고 주장하며 긍정적인 전망을 제시합니다.
*   블랙웰의 핵심 혁신은 기존의 텍스트 기반 AI를 넘어 비디오를 '섭취'하고 학습하는 능력으로, 이는 AI 기술의 '차세대 개척지'로 평가됩니다.
*   이러한 비디오 기반 학습 능력은 로봇 공학 발전의 핵심 동력으로, 로봇이 관찰을 통해 인간과 유사한 정교한 행동을 모방하고 학습하게 만듭니다.
*   미래의 로봇은 단순한 기계를 넘어, 사용자의 과거 이력이나 상황을 기억하여 '파트너'처럼 자연스럽고 개인화된 상호작용을 제공할 것입니다.
*   이러한 장기적인 기술 혁신 전망에도 불구하고, 전반적인 시장 하락세의 영향으로 단기적인 주가 하락은 불가피할 것으로 예상됩니다.


## <font color=yellow>3. OpenAI TTS API를 이용하여 음성 요약하기

In [18]:
from openai import OpenAI
client = OpenAI()

In [19]:
speech_file_path = "./outputs/audio_summary.wav"

response = client.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input=final_summary
)

response.stream_to_file(speech_file_path)

C:\Users\jjsd4\AppData\Local\Temp\ipykernel_8348\2395740819.py:9: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


In [20]:
from IPython.display import Audio

# Audio 객체를 생성하고 출력합니다.
audio = Audio(speech_file_path, autoplay=False)
audio

## <font color=yellow>4. LLM을 활용하여 요약된 정보들을 취합하기</font>
- 이전 summary는 문서형으로 이루어져 있었고, 팟캐스트로 듣기 좋은 형태로 만들어보겠음

In [22]:
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

In [24]:
llm = ChatOpenAI(model='gpt-4o')

prompt_template = ChatPromptTemplate.from_messages(
        [("user", "{summary}\n위 내용을 전달하기 위해 사용자가 듣기 편한 짧막한 오디오 대본을 만들어줘. 너의 응답을 그대로 TTS에 넣을 것이니 자연스러운 발화만 이야기해줘. 뉴스를 전하는 톤으로 존댓말을 사용해서 만들어줘.")]
    )

natural_summary_chain = prompt_template | llm | StrOutputParser()

natural_summary = natural_summary_chain.invoke({"summary": final_summary})

In [25]:
natural_summary

"안녕하세요, 오늘의 주요 소식 전해드립니다.\n\n첫째, 한 분석가는 엔비디아의 차세대 칩 '블랙웰'이 조기 출시되면서, 높은 수익이 기대됨에 따라 현재 주가가 저평가되었다고 긍정적으로 평가했습니다.\n\n둘째, 블랙웰은 기존의 텍스트 기반 AI를 넘어 비디오를 분석하고 학습하는 능력을 갖추어, 차세대 AI 기술의 개척지로 주목받고 있습니다.\n\n셋째, 비디오 기반 학습 능력은 로봇 공학의 발전을 이끌며, 로봇이 인간처럼 정교한 행동을 관찰하고 모방할 수 있게 합니다.\n\n넷째, 미래의 로봇은 단순한 기계를 넘어, 사용자의 과거 이력과 상황을 기억하여 보다 자연스럽고 개인화된 상호작용을 제공할 것으로 기대됩니다.\n\n마지막으로, 이러한 기술 혁신 전망에도 불구하고, 시장의 전반적인 하락세 때문에 단기적인 주가 하락은 불가피할 것으로 보입니다.\n\n이상, 오늘의 주요 소식이었습니다. 감사합니다."

In [26]:
speech_file_path = "./outputs/audio_natural_summary.wav"

response = client.audio.speech.create(
  model="tts-1",
  voice="alloy",
  input=natural_summary
)

response.stream_to_file(speech_file_path)

C:\Users\jjsd4\AppData\Local\Temp\ipykernel_8348\1539680276.py:9: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


In [27]:
from IPython.display import Audio

# Audio 객체를 생성하고 출력합니다.
audio = Audio(speech_file_path, autoplay=False)
audio